# This will be the VacationPy sub-challenge

In [1]:
# Import dependencies, api_key and configure gmaps with api_key
import gmaps
import pandas as pd
from api_keys import gkey
import random as r
import numpy as np
import requests
gmaps.configure(api_key=gkey)

In [2]:
# Load the DataFrame from WeatherPy
weather_data = pd.read_csv('../WeatherPy/Output/weather_data.csv')
weather_data.head()

,city,latitude (°N),longitude (°E),temperature (°F),humidity (%),cloudiness (%),wind speed (mph)
0,"Tandur, IN",19.151955,79.448860,72.90,94,100,13.89
1,"Kapaa, US",22.074733,-159.319160,83.93,62,20,9.22
2,"Puerto Ayora, EC",-0.747167,-90.313420,70.20,93,97,7.00
3,"Kalispell, US",48.202158,-114.315321,84.76,20,0,5.75
4,"Codrington, AG",17.640463,-61.825520,82.49,78,87,10.87


In [3]:
# Build heat map layer with info from df
heatmap = gmaps.heatmap_layer(weather_data[['latitude (°N)', 'longitude (°E)']], weights=weather_data['humidity (%)'], max_intensity=100, point_radius=10)
ideal_vacation_map = gmaps.figure()
ideal_vacation_map.add_layer(heatmap)

# Show heat map
ideal_vacation_map

In [5]:
# Narrow down weather to ideal conditions
ideal_weather = weather_data.loc[(np.abs(weather_data['temperature (°F)']-75)<=5)&(weather_data['wind speed (mph)']<5)&(weather_data['cloudiness (%)']==0),:].reset_index(drop=True)
ideal_weather

In [7]:
# Create a function to search location for a hotel using google places api
def get_closest_hotel(lat, lon):

    # Base URL for places api + parameters
    places_url = f'https://maps.googleapis.com/maps/api/place/nearbysearch/json?key={gkey}&radius=5000&type=lodging&location={lat},{lon}'
    
    # Call api and access the first result '[0]'
    first_response = requests.get(places_url).json()['results'][0]
    
    # Package and return the results as a dictionary 
    return {'hotel': first_response['name'], 'latitude': first_response['geometry']['location']['lat'], 'longitude': first_response['geometry']['location']['lng']}

In [8]:
# Intialize 2 lists
locations = []
hotel_info_list = []
for lat, lon, city in zip(ideal_weather['latitude (°N)'], ideal_weather['longitude (°E)'], ideal_weather['city']):
    hotel = get_closest_hotel(lat, lon)
    locations.append((hotel['latitude'], hotel['longitude']))
    hotel_info = f"Hotel: {hotel['hotel']} City: {city.split()[0]} Country: {city.split()[1]}"
    hotel_info_list.append(hotel_info)

# Create the markers and show the figure
hotel_markers = gmaps.marker_layer(locations=locations, info_box_content=hotel_info_list)
ideal_vacation_map.add_layer(hotel_markers)
ideal_vacation_map